In [51]:
import random
from random import randint as rand

In [52]:
totalPopulation = 150
alpha_list = [x for x in range(0, 8)]

In [53]:
def checkInc_x_Inc_y(x, y, randomData):
    list_non_attack_queens = []
    while x <=7 and y <= 7:
        isAttack = 0
        #print('Checking if any queen exists in x : '+str(x) + 'and y : '+str(y))
        isAttack = y == randomData[x]
        #print(isAttack)
        if isAttack == 1:
            x += 1
            y += 1
            continue
        else:
            list_non_attack_queens.append(x)
            x += 1
            y += 1
    list_non_attack_queens += [x1 for x1 in range(x, 8)]
    return list_non_attack_queens

In [54]:
def checkInc_x_Dec_y(x, y, randomData):
    list_non_attack_queens = []
    while x <=7 and y >= 0:
        isAttack = 0
        #print('Checking if any queen exists in x : '+str(x) + 'and y : '+str(y))
        isAttack = y == randomData[x]
        #print(isAttack)
        if isAttack == 1:
            x += 1
            y -= 1
            continue
        else:
            list_non_attack_queens.append(x)
            x += 1
            y -= 1
    list_non_attack_queens += [x1 for x1 in range(x, 8)]
    return list_non_attack_queens


In [55]:
def checkInc_x_Cons_y(x, y, randomData):
    list_non_attack_queens = []
    while x <=7:
        isAttack = 0
        #print('Checking if any queen exists in x : '+str(x) + 'and y : '+str(y))
        isAttack = y == randomData[x]
        #print(isAttack)
        if isAttack == 1:
            x += 1
            continue
        else:
            list_non_attack_queens.append(x)
            x += 1
    list_non_attack_queens += [x1 for x1 in range(x, 8)]
    return list_non_attack_queens

In [56]:
def getFitnessScore(randomData):
    fitnessScore = 0
    #Iterate through each of the queens position and check if it attacks any other queen
    for idx, queen in enumerate(randomData):
        # Check if there is any other queen within the same Row
        # print('Checking Queen Position x : '+str(idx) +'and y : ' +str(queen))
        # Check Diagonality constraint of this queen with others
        isAttack_Inc_x_Inc_y = set(checkInc_x_Inc_y(idx+1, queen+1, randomData))
        isAttack_Inc_x_Dec_y = set(checkInc_x_Dec_y(idx+1, queen-1, randomData))
        isAttack_Inc_x_Cons_y = set(checkInc_x_Cons_y(idx+1, queen, randomData))
        #isAttack_Dec_x_Inc_y = checkDec_x_Inc_y(idx-1, queen+1, randomData)
        #isAttack_Dec_x_Dec_y = checkDec_x_Dec_y(idx-1, queen-1, randomData)
        list_of_non_attack_queens = [isAttack_Inc_x_Inc_y, isAttack_Inc_x_Dec_y, isAttack_Inc_x_Cons_y]
        non_empty_sets = [x for x in list_of_non_attack_queens if x]
        if len(non_empty_sets) == 3:
            non_attack_queens = set.intersection(*non_empty_sets)
            fitnessScore += len(non_attack_queens)
        # print('Queen Position for Column '+str(idx) +'---' +str(queen) +' and its fitnessScore is '+str(fitnessScore))
    return fitnessScore

In [57]:
populationData = []
fitnessData = []
secure_random = random.SystemRandom()
'''for outloop in range(totalPopulation):
  randomData = []
  fitnessScore = 0
  for inloop in range(8):
    selectedData = secure_random.choice(alpha_list)
    randomData.append(selectedData)
  #randomData = [1,4,3,0,2,5,6,0]
  populationData.append(randomData)
  fitnessScore = getFitnessScore(randomData)
  fitnessData.append(fitnessScore)'''
for outloop in range(totalPopulation):
    randomData = {"dna": [0, 0, 0, 0, 0, 0, 0, 0], "fitnessScore": 0, "prob": 0.0}
    fitnessScore = 0
    dna = []
    for inloop in range(8):
        selectedData = secure_random.choice(alpha_list)
        dna.append(selectedData)
    #randomData = [1,4,3,0,2,5,6,0]
    randomData["dna"] = dna
    randomData["fitnessScore"] = getFitnessScore(dna)
    populationData.append(randomData)
    #fitnessScore = getFitnessScore(randomData)
    #fitnessData.append(fitnessScore)
totalFitnessScore = 0
for outloop in range(len(populationData)):
    totalFitnessScore += populationData[outloop]["fitnessScore"]
for outloop in range(len(populationData)):
    populationData[outloop]["prob"] = populationData[outloop]["fitnessScore"] / totalFitnessScore

In [58]:
def selection(populationData):
    index = 0
    r = random.random()
    while r > 0:
        r = r - populationData[index]["prob"]
        index += 1
    index -= 1
    return populationData[index]["dna"].copy()

In [59]:
def crossover_and_mutate():
    loop = 0
    flag = 0
    while True:
        print('Generation : ', loop, '--------------------------------------------------------------------------------')
        partnerA = selection(populationData)
        partnerB = selection(populationData)
        partnerC = selection(populationData)
        partnerD = selection(populationData)
        partnerA[:3], partnerB[3:] = partnerB[:3], partnerA[3:]
        partnerC[:3], partnerD[3:] = partnerD[:3], partnerC[3:]

        new_population = [partnerA, partnerB, partnerC, partnerD]
        for population in new_population:
            # Check for except between alpha_list and population
            new_alpha_list = list(set(alpha_list).difference(set(population)))
            dict = {}
            new_idx = 0
            for idx, queen_pos in enumerate(population):
                if queen_pos in dict.keys():
                    population[idx] = new_alpha_list[new_idx]
                    new_idx += 1
                else:
                    dict[queen_pos] = queen_pos
            index_mutant_pos1 = rand(0, 7)
            index_mutant_pos2 = rand(0, 7)
            population[index_mutant_pos1], population[index_mutant_pos2] = population[index_mutant_pos2], \
                                                                           population[index_mutant_pos1]
        populationData.append({"dna": new_population[0], "fitnessScore": getFitnessScore(new_population[0]), "prob": 0.0})
        populationData.append({"dna": new_population[1], "fitnessScore": getFitnessScore(new_population[1]), "prob": 0.0})
        populationData.append({"dna": new_population[2], "fitnessScore": getFitnessScore(new_population[2]), "prob": 0.0})
        populationData.append({"dna": new_population[3], "fitnessScore": getFitnessScore(new_population[3]), "prob": 0.0})
        print('childA : ', new_population[0], 'and fitnessScore : ', getFitnessScore(new_population[0]))
        print('childB : ', new_population[1], 'and fitnessScore : ', getFitnessScore(new_population[1]))
        print('childC : ', new_population[2], 'and fitnessScore : ', getFitnessScore(new_population[2]))
        print('childD : ', new_population[3], 'and fitnessScore : ', getFitnessScore(new_population[3]))
        print('----------------------------------------End of Generation----------------------------------------------')
        # Recalculate the Probability with the new Child Elements added to the population
        totalFitnessScore = 0
        for outloop in range(len(populationData)):
            totalFitnessScore += populationData[outloop]["fitnessScore"]
        for outloop in range(len(populationData)):
            populationData[outloop]["prob"] = populationData[outloop]["fitnessScore"] / totalFitnessScore
            if populationData[outloop]["fitnessScore"] == 28:
                print('Optimal Position for 8 Queens Problem: ', populationData[outloop]["dna"])
                flag = 1
        if flag == 1:
            break
        loop += 1

In [60]:
crossover_and_mutate()

Generation :  0 --------------------------------------------------------------------------------
childA :  [1, 7, 0, 5, 4, 6, 3, 2] and fitnessScore :  23
childB :  [1, 4, 0, 7, 5, 6, 3, 2] and fitnessScore :  22
childC :  [1, 6, 3, 4, 7, 0, 2, 5] and fitnessScore :  23
childD :  [3, 6, 1, 0, 7, 4, 2, 5] and fitnessScore :  23
----------------------------------------End of Generation----------------------------------------------
Generation :  1 --------------------------------------------------------------------------------
childA :  [4, 2, 0, 1, 7, 6, 3, 5] and fitnessScore :  22
childB :  [4, 3, 6, 1, 7, 0, 2, 5] and fitnessScore :  22
childC :  [2, 1, 5, 0, 4, 3, 6, 7] and fitnessScore :  20
childD :  [3, 1, 5, 0, 4, 2, 7, 6] and fitnessScore :  22
----------------------------------------End of Generation----------------------------------------------
Generation :  2 --------------------------------------------------------------------------------
childA :  [6, 2, 7, 3, 0, 1, 4, 5] an